# AUTOMATED ATTENDANCE SYSTEM

                                                    MADE BY:
                                                    
                                               -Shivam (1510991611)     
                                               -Tanmay (1510991673)
                                               -Tanya (1510991677)
                                               -Vishal (1510991730)
                                               
                                               
                                                                       ...in chronogical order of the respective roll numbers

## BUILDING THE DATABASE

The following chunk of code was used to build the database, with its schema.

## IMPORTS ##

As a pre-execution step, we have to decide which libraries we have to use for execution.

1. We have used SQLAlchemy for an Object Relational Mapping of the table in our database.
2. We build a session and create a sqlite engine as connection for the table.

- We import sklearn for KNNClassifier, which is the backbone of our algorithm.
- We import PIL for imagery.
- We import face_recognition (a dlib based interface for face_recognition). It has all the features of OpenCV face recognition and more, with support for many ways to do so.

In [12]:
from sqlalchemy import create_engine
from sqlalchemy.orm import sessionmaker

from db import Attendance, Base

engine = create_engine('sqlite:///attendance.db')
Base.metadata.bind = engine
DBSession = sessionmaker(bind=engine)
session = DBSession()

In [13]:
import glob
import math
from sklearn import neighbors
import os
import os.path
import pickle
from PIL import Image, ImageDraw
import face_recognition
from face_recognition.face_recognition_cli import image_files_in_folder

We allow image type to be either of png or jpeg format.

In [14]:
ALLOWED_EXTENSIONS = {'png', 'jpg', 'jpeg'}

## Training the face detection classifier

Now, coming to training the classifier:
face_recognition, one of the libraries used works for a heirarchy of folders.

The train directory contains the subdirectories labeled as the names of the individuals. We have used _ as a seperator between names and roll numbers.
The subdirectories contain the images, on which there is no labeling constraint.

Trains a k-nearest neighbors classifier for face recognition.

     Structure:
        <train_dir>/
        ├── <person1>/
        │   ├── <somename1>.jpeg
        │   ├── <somename2>.jpeg
        │   ├── ...
        ├── <person2>/
        │   ├── <somename1>.jpeg
        │   └── <somename2>.jpeg
        └── ...




- param train_dir: directory that contains a sub-directory for each known person, with its name.

- param model_save_path: (optional) path to save model on disk

- param n_neighbors: (optional) number of neighbors to weigh in classification. Chosen automatically if not specified

- param knn_algo: (optional) underlying data structure to support knn.default is ball_tree

- param verbose: verbosity of training

- return: returns knn classifier that was trained on the given data.

We:
- Loop through each person in the training set
- Loop through each training image for the current person
- If there are no people (or too many people) in a training image, skip the image.
- Add face encoding for current image to the training set
- Determine how many neighbors to use for weighting in the KNN classifier
- Create and Save the trained KNN classifier
    

In [15]:
def train(train_dir, model_save_path=None, n_neighbors=None, knn_algo='ball_tree', verbose=False):
    X = []
    y = []

    for class_dir in os.listdir(train_dir):
        if not os.path.isdir(os.path.join(train_dir, class_dir)):
            continue

        for img_path in image_files_in_folder(os.path.join(train_dir, class_dir)):
            image = face_recognition.load_image_file(img_path)
            face_bounding_boxes = face_recognition.face_locations(image)

            if len(face_bounding_boxes) != 1:
                if verbose:
                    print("Image {} not suitable for training: {}".format(img_path, "Didn't find a face" if len(face_bounding_boxes) < 1 else "Found more than one face"))
            else:
                X.append(face_recognition.face_encodings(image, known_face_locations=face_bounding_boxes)[0])
                y.append(class_dir)

    if n_neighbors is None:
        n_neighbors = int(round(math.sqrt(len(X))))
        if verbose:
            print("Chose n_neighbors automatically:", n_neighbors)

    knn_clf = neighbors.KNeighborsClassifier(n_neighbors=n_neighbors, algorithm=knn_algo, weights='distance')
    knn_clf.fit(X, y)

    if model_save_path is not None:
        with open(model_save_path, 'wb') as f:
            pickle.dump(knn_clf, f)

    return knn_clf

## FROM NEAREST NEIGHBOUR MODELLING TO MAKING PREDICTIONS



Recognizes faces in given image using a trained KNN classifier

    :param X_img_path: path to image to be recognized
    :param knn_clf: (optional) a knn classifier object. if not specified, model_save_path must be specified.
    :param model_path: (optional) path to a pickled knn classifier. if not specified, model_save_path must be knn_clf.
    :param distance_threshold: (optional) distance threshold for face classification. the larger it is, the more chance
           of mis-classifying an unknown person as a known one.
    :return: a list of names and face locations for the recognized faces in the image: [(name, bounding box), ...].
        For faces of unrecognized persons, the name 'unknown' will be returned.

Here, we:
- Load a trained KNN model (if one was passed in)
- Load image file and find face locations
- If no faces are found in the image, return an empty result.
- Find encodings for faces in the test iamge
- Use the KNN model to find the best matches for the test face
- Predict classes and remove classifications that aren't within the threshold
    

In [16]:
def predict(X_img_path, knn_clf=None, model_path=None, distance_threshold=0.6):
    if not os.path.isfile(X_img_path) or os.path.splitext(X_img_path)[1][1:] not in ALLOWED_EXTENSIONS:
        raise Exception("Invalid image path: {}".format(X_img_path))

    if knn_clf is None and model_path is None:
        raise Exception("Must supply knn classifier either thourgh knn_clf or model_path")

    if knn_clf is None:
        with open(model_path, 'rb') as f:
            knn_clf = pickle.load(f)

    X_img = face_recognition.load_image_file(X_img_path)
    X_face_locations = face_recognition.face_locations(X_img)

    if len(X_face_locations) == 0:
        return []

    faces_encodings = face_recognition.face_encodings(X_img, known_face_locations=X_face_locations)

    closest_distances = knn_clf.kneighbors(faces_encodings, n_neighbors=1)
    are_matches = [closest_distances[0][i][0] <= distance_threshold for i in range(len(X_face_locations))]

    return [(pred, loc) if rec else ("unknown", loc) for pred, loc, rec in zip(knn_clf.predict(faces_encodings), X_face_locations, are_matches)]


## SHOW LABELS

Shows the face recognition results visually.

:param img_path: path to image to be recognized
:param predictions: results of the predict function
:return:

- Draw a box around the face using the Pillow module
- There's a bug in Pillow where it blows up with non-UTF-8 text when using the default bitmap font
- Draw a label with a name below the face
- Remove the drawing library from memory as per the Pillow docs
- Display the resulting image
     

In [17]:
def show_prediction_labels_on_image(img_path, predictions):
    
    pil_image = Image.open(img_path).convert("RGB")
    draw = ImageDraw.Draw(pil_image)

    for name, (top, right, bottom, left) in predictions:
        draw.rectangle(((left, top), (right, bottom)), outline=(0, 0, 255))

        name = name.encode("UTF-8")

        text_width, text_height = draw.textsize(name)
        draw.rectangle(((left, bottom - text_height - 10), (right, bottom)), fill=(0, 0, 255), outline=(0, 0, 255))
        draw.text((left + 6, bottom - text_height - 5), name, fill=(255, 255, 255, 255))

    del draw

    pil_image.show()


## COMBINING IT ALL: EXECUTION

#### STEP 1: Train the KNN classifier and save it to disk
 Once the model is trained and saved, you can skip this step next time.
#### STEP 2: Using the trained classifier, make predictions for unknown images
  Find all people in the image using a trained classifier model
  Note: You can pass in either a classifier file name or a classifier model instance
  Print results on the console
#### STEP 3: Show labeled predictions on  images 
             

In [19]:
if __name__ == "__main__":
    print("Training KNN classifier...")
    classifier = train("knn_examples/train", model_save_path="trained_knn_model.clf", n_neighbors=2)
    print("Training complete!")

    for image_file in os.listdir("knn_examples/test"):
        full_file_path = os.path.join("knn_examples/test", image_file)

        print("Looking for faces in {}".format(image_file))

        predictions = predict(full_file_path, model_path="trained_knn_model.clf")
        
        preds=[]
        for name, (top, right, bottom, left) in predictions:
            preds.append(name)
            print("- Found {} at ({}, {})".format(name, left, top))
        print(preds)
        
        students  = []
        present = []
        
        students_db = session.query(Attendance).all()
        for s in students_db:
            students.append(s.roll_no)
        print(students)
        
        for i in preds:
            _= i.split('_')[1]
            present.append(_)
        print(present)
        
        for i in students:
            
            if i in present:
                #print('Present',i)
                x = session.query(Attendance).filter_by(roll_no=i).first()
                
                x.attend = 1
                
                if(x.attend)>=1:
                    x.attend += 1
                session.add(x)
                #s1 = Attendance(attend='1',roll_no=i)
                #session.add(s1)
        session.commit()
        show_prediction_labels_on_image(os.path.join("knn_examples/test", image_file), predictions)

Training KNN classifier...
Training complete!
Looking for faces in WIN_20180519_01_57_50_Pro.jpg
- Found navroop_1510991401 at (706, 142)
- Found tanmay_1510991673 at (324, 325)
['navroop_1510991401', 'tanmay_1510991673']
[]
['1510991401', '1510991673']
Looking for faces in WIN_20180519_10_46_10_Pro (2).jpg
- Found tanmay_1510991673 at (588, 316)
- Found shivam_1510991611 at (902, 242)
- Found vishal_1510991730 at (411, 225)
['tanmay_1510991673', 'shivam_1510991611', 'vishal_1510991730']
[]
['1510991673', '1510991611', '1510991730']
Looking for faces in WIN_20180519_11_10_37_Pro.jpg
- Found tanmay_1510991673 at (772, 235)
- Found vishal_1510991730 at (562, 270)
- Found shivam_1510991611 at (268, 282)
- Found tanya_1510991677 at (1010, 305)
['tanmay_1510991673', 'vishal_1510991730', 'shivam_1510991611', 'tanya_1510991677']
[]
['1510991673', '1510991730', '1510991611', '1510991677']
Looking for faces in WIN_20180519_13_32_49_Pro.jpg
- Found tanmay_1510991673 at (266, 266)
- Found mam at 

IndexError: list index out of range

## CONCLUSION


Through this, we can conclude that:
- It is fairly easy to o face detection and recognition, with a pre-existing database for trained and testing images.
- KNN uses a nearest neighbour approach, which helps as a metric (distance), from which we can deduce the accuracy of the match between the trained and tested image, with the same detection.
- This is an intelligent approach for feature detection and recognition, in which, we compare the face encodings of the two images to be compared. 


The face encodings are an appropriate measure to compare the most important features of the cropped face (enclosed within the bounding box), and they are compared using a measure of _'euclidean distance'_.